In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os

In [12]:
cv = pd.read_csv(r"D:\NUS\project\LTA_data\comuting_flow\commuting_flow_202205.csv")
cv.head(5)

,ind,TOTAL_TRIPS,od_pairs,origin_stop,destination_stop,Origin_Subzone,Destination_Subzone
0,0,2552,"(1012, 1112)",1012,1112,VICTORIA,VICTORIA
1,1,2361,"(1012, 1113)",1012,1113,VICTORIA,BUGIS
2,2,1467,"(1012, 1121)",1012,1121,VICTORIA,ROCHOR CANAL
3,3,1694,"(1012, 1211)",1012,1211,VICTORIA,CRAWFORD
4,4,2771,"(1012, 1311)",1012,1311,VICTORIA,LAVENDER


In [13]:
cv_subzones = cv.groupby(['Origin_Subzone','Destination_Subzone']).sum('TOTAL_TRIPS')
cv_subzones.head()

ind  TOTAL_TRIPS  origin_stop  \
Origin_Subzone Destination_Subzone                                       
ADMIRALTY      ADMIRALTY            12575931        14469      3553691   
               BALESTIER              620622          235       174993   
               BOULEVARD              413723          235       116662   
               BRAS BASAH             413712           95       116662   
               BUKIT MERAH            413732           15       116662   

                                    destination_stop  
Origin_Subzone Destination_Subzone                    
ADMIRALTY      ADMIRALTY                     3552037  
               BALESTIER                      150145  
               BOULEVARD                       18266  
               BRAS BASAH                      12248  
               BUKIT MERAH                     20090

In [14]:
cv_subzones = cv_subzones.reset_index()
cv_subzones

,Origin_Subzone,Destination_Subzone,ind,TOTAL_TRIPS,origin_stop,destination_stop
0,ADMIRALTY,ADMIRALTY,12575931,14469,3553691,3552037
1,ADMIRALTY,BALESTIER,620622,235,174993,150145
2,ADMIRALTY,BOULEVARD,413723,235,116662,18266
3,ADMIRALTY,BRAS BASAH,413712,95,116662,12248
4,ADMIRALTY,BUKIT MERAH,413732,15,116662,20090
...,...,...,...,...,...,...
24491,YUNNAN,TUKANG,842940,542,219190,177352
24492,YUNNAN,WENYA,3953125,31102,1010306,1042628
24493,YUNNAN,YUHUA EAST,5477917,4592,1386335,1494849
24494,YUNNAN,YUHUA WEST,7029799,16746,1777762,1935038


In [15]:
cv_subzones = cv_subzones[['Origin_Subzone','Destination_Subzone','TOTAL_TRIPS']]
cv_subzones

,Origin_Subzone,Destination_Subzone,TOTAL_TRIPS
0,ADMIRALTY,ADMIRALTY,14469
1,ADMIRALTY,BALESTIER,235
2,ADMIRALTY,BOULEVARD,235
3,ADMIRALTY,BRAS BASAH,95
4,ADMIRALTY,BUKIT MERAH,15
...,...,...,...
24491,YUNNAN,TUKANG,542
24492,YUNNAN,WENYA,31102
24493,YUNNAN,YUHUA EAST,4592
24494,YUNNAN,YUHUA WEST,16746


In [18]:
lis_origin_subzone = np.unique(list(set(cv_subzones['Origin_Subzone'])))

In [19]:
len(lis_origin_subzone)

313

In [20]:
subzone = gpd.read_file(r"D:\NUS\project\Singapore_subzone\subzone_2019\MasterPlan_SUBZONE_2019\master-plan-2019-subzone-boundary-no-sea-geojson-polygon.shp")
subzone = subzone.to_crs('epsg:4326')
lis_subzone = np.unique(list(set(subzone['SUBZONE_N'])))
len(lis_subzone)

332

In [24]:
cols = ['SUBZONE']
for i in lis_subzone:
    cols.append(i)
len(cols)

333

In [58]:
cf_matrix = pd.DataFrame(columns = cols)
cf_matrix

,SUBZONE,ADMIRALTY,AIRPORT ROAD,ALEXANDRA HILL,ALEXANDRA NORTH,ALJUNIED,ANAK BUKIT,ANCHORVALE,ANG MO KIO TOWN CENTRE,ANSON,...,YIO CHU KANG EAST,YIO CHU KANG NORTH,YIO CHU KANG WEST,YISHUN CENTRAL,YISHUN EAST,YISHUN SOUTH,YISHUN WEST,YUHUA EAST,YUHUA WEST,YUNNAN


In [67]:
def output_destination(subzone,df):
    dataframe = df.loc[df['Origin_Subzone'] == subzone].copy()
    dataframe.index = range(len(dataframe))
    lis_destination = dataframe.loc[:,'Destination_Subzone']
    lis_total_trip = dataframe.loc[:,'TOTAL_TRIPS']
    
    return lis_destination,lis_total_trip

In [84]:
def commuting_flow_matrix(subzone,lis_sz,lis_trip):
    m = pd.DataFrame(columns = cols)
    m.loc[0,'SUBZONE'] = subzone
    for i in range(len(lis_subzone)):
        for x in range(len(lis_sz)):
            if lis_sz[x] == lis_subzone[i]:
                m.loc[0,lis_subzone[i]] = lis_trip[x]               
    return m

In [85]:
lis_sz,lis_trip = output_destination('ADMIRALTY',cv_subzones)
mx = commuting_flow_matrix('ADMIRALTY',lis_sz,lis_trip)
mx

,SUBZONE,ADMIRALTY,AIRPORT ROAD,ALEXANDRA HILL,ALEXANDRA NORTH,ALJUNIED,ANAK BUKIT,ANCHORVALE,ANG MO KIO TOWN CENTRE,ANSON,...,YIO CHU KANG EAST,YIO CHU KANG NORTH,YIO CHU KANG WEST,YISHUN CENTRAL,YISHUN EAST,YISHUN SOUTH,YISHUN WEST,YUHUA EAST,YUHUA WEST,YUNNAN
0,ADMIRALTY,14469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,19538,217,2288,31854,NaN,NaN,NaN


In [93]:
matrix = pd.DataFrame(columns = cols)
for i in lis_subzone:
    lis_sz,lis_trip = output_destination(i,cv_subzones)
    mx = commuting_flow_matrix(i,lis_sz,lis_trip)
    matrix = matrix.append(mx)
matrix

C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.ap

C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.ap

C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.ap

C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.ap

C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.ap

C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.ap

C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.ap

C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.ap

C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matrix = matrix.append(mx)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_3004/1429397337.py:5: FutureWarning: The frame.ap

,SUBZONE,ADMIRALTY,AIRPORT ROAD,ALEXANDRA HILL,ALEXANDRA NORTH,ALJUNIED,ANAK BUKIT,ANCHORVALE,ANG MO KIO TOWN CENTRE,ANSON,...,YIO CHU KANG EAST,YIO CHU KANG NORTH,YIO CHU KANG WEST,YISHUN CENTRAL,YISHUN EAST,YISHUN SOUTH,YISHUN WEST,YUHUA EAST,YUHUA WEST,YUNNAN
0,ADMIRALTY,14469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,19538,217,2288,31854,NaN,NaN,NaN
0,AIRPORT ROAD,NaN,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,ALEXANDRA HILL,NaN,NaN,39415,1348,1800,1567,NaN,NaN,1516,...,NaN,NaN,NaN,1352,NaN,NaN,37,1005,1079,NaN
0,ALEXANDRA NORTH,NaN,NaN,950,1969,309,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,ALJUNIED,NaN,NaN,2100,NaN,316952,643,1149,2427,1098,...,351,NaN,1126,285,NaN,NaN,NaN,83,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,YISHUN SOUTH,NaN,NaN,1332,NaN,66,635,7137,10844,NaN,...,124,24728,3468,168439,72120,104736,42027,NaN,NaN,NaN
0,YISHUN WEST,27676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,173045,59313,66675,351146,NaN,NaN,NaN
0,YUHUA EAST,NaN,NaN,634,NaN,NaN,1848,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25650,53828,4406
0,YUHUA WEST,NaN,NaN,1313,NaN,NaN,4407,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47880,43191,14436


In [96]:
matrix = matrix.fillna(0)
matrix.to_csv(r'D:\NUS\project\LTA_data\comuting_flow\subzone_commuting_flow_matrix.csv',index_label = 'ind')